# Exportar Dataset para Machine Learning
Este script extrae la tabla final `analytics.daily_features` y la guarda como 
un archivo CSV. Este archivo CSV será el **input oficial** para tus futuros 
modelos de ML.

In [5]:
import os
import pandas as pd
from sqlalchemy import create_engine

# 1. Configurar conexión 
PG_USER = os.getenv('PG_USER', 'admin')
PG_PASSWORD = os.getenv('PG_PASSWORD', 'admin_password')
PG_HOST = os.getenv('PG_HOST', 'postgres')
PG_DB = os.getenv('PG_DB', 'trading_db')

db_url = f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:5432/{PG_DB}"
engine = create_engine(db_url)

# 2. Descargar toda la tabla
# Ordenamos por fecha y ticker para mantener el orden temporal, vital en series de tiempo.
query = "SELECT * FROM analytics.daily_features ORDER BY date ASC, ticker ASC"

print("Leyendo datos desde Postgres...")
df_full = pd.read_sql(query, engine)

print(f"Dimensiones del Dataset: {df_full.shape}")
print("Columnas:", df_full.columns.tolist())

# 3. Visualizar los datos
print("\n--- Primeras 5 filas ---")
print(df_full.head())

# Verificar que no tenemos nulos críticos en los features que usaremos
print("\n--- Conteo de Nulos ---")
print(df_full[['return_close_open', 'return_prev_close','volatility_20_days']].isnull().sum())

# 4. Guardar a CSV
output_dir = "./" 

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_file = os.path.join(output_dir, "dataset_ml_features.csv")

# Guardar
df_full.to_csv(output_file, index=False)

print(f"\nArchivo guardado exitosamente en: {output_file}")
print("Ahora puedes usar este archivo para entrenar tus modelos.")

Leyendo datos desde Postgres...
Dimensiones del Dataset: (4458, 15)
Columnas: ['date', 'ticker', 'year', 'month', 'day_of_week', 'open', 'close', 'high', 'low', 'volume', 'return_close_open', 'return_prev_close', 'volatility_20_days', 'run_id', 'ingested_at_utc']

--- Primeras 5 filas ---
         date ticker  year  month  day_of_week        open       close  \
0  2020-01-02    GLD  2020      1            3  143.860001  143.949997   
1  2020-01-02    QQQ  2020      1            3  214.399994  216.160004   
2  2020-01-02    SPY  2020      1            3  323.540009  324.869995   
3  2020-01-03    GLD  2020      1            4  145.750000  145.860001   
4  2020-01-03    QQQ  2020      1            4  213.300003  214.179993   

         high         low    volume  return_close_open  return_prev_close  \
0  144.210007  143.399994   7733800           0.000626           0.000000   
1  216.160004  213.979996  30969400           0.008209           0.000000   
2  324.890015  322.529999  5915120